---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [134]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    #P1_Graphs[0].graph
    
    return ['PA','SW_L','SW_L','PA','SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [39]:
def salary_predictions():
    
    #create df out of G network
    employee_id = [x for x,y in G.nodes(data=True)]
    department = [y['Department'] for x,y in G.nodes(data=True)]
    m_salary = [y['ManagementSalary'] for x,y in G.nodes(data=True)]
    G_df = pd.DataFrame({'employee_id':employee_id,
                         'department':department,
                         'm_salary':m_salary})

    #create features out of networkx
    G_df['degree'] = nx.degree(G).values()
    G_df['closeness_centrality'] = nx.closeness_centrality(G).values()
    G_df['betweenness_centrality'] = nx.betweenness_centrality(G).values()
    G_df['page_rank'] = nx.pagerank(G).values()
    G_df['hub'] = nx.hits(G)[0].values()
    G_df['authority'] = nx.hits(G)[1].values()
    #create dummy variable for department
    dummies = pd.get_dummies(G_df['department'], prefix = 'dept')
    G_df = G_df.join(dummies)

    #split data into train and test
    test = G_df[G_df['m_salary'].isnull()==1]
    train = G_df[G_df['m_salary'].isnull()==0]
    #look at feature dist for different values of target variable
    #for i in train.iloc[:,3:9].columns:
        #print(train[train['m_salary']==0][i].mean(),train[train['m_salary']==1][i].mean())
        
    #split train data into X and y
    test_index = G_df[G_df['m_salary'].isnull()]['employee_id']
    train_y = train['m_salary']
    train_X = train.drop(['m_salary','department','employee_id'], axis = 1)
    test_X = test.drop(['m_salary','department','employee_id'], axis=1)

    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import roc_curve, auc
    gbclf = GradientBoostingClassifier().fit(train_X, train_y)
    pred_out = gbclf.predict_proba(test_X)

    pred_output = pd.DataFrame(pred_out, index = test_index)
    result = pd.Series(pred_output[1])

    # #build a model
    # %matplotlib inline
    # from sklearn.ensemble import GradientBoostingClassifier
    # from sklearn.model_selection import train_test_split
    # from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    # from sklearn.metrics import roc_curve, auc
    # import matplotlib.pyplot as plt


    # X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, random_state = 0)
    # gbclf = GradientBoostingClassifier().fit(X_train, y_train)
    # pred_output = gbclf.predict(X_test)
    # y_score_gbclf = gbclf.decision_function(X_test)
    # fpr_gbclf, tpr_gbclf, _ = roc_curve(y_test, y_score_gbclf)
    # roc_auc_gbclf = auc(fpr_gbclf, tpr_gbclf)

    # plt.figure()
    # plt.xlim([-0.01, 1.00])
    # plt.ylim([-0.01, 1.01])
    # plt.plot(fpr_gbclf, tpr_gbclf, lw=3, label='gbclf ROC curve (area = {:0.2f})'.format(roc_auc_gbclf))
    # plt.xlabel('False Positive Rate', fontsize=16)
    # plt.ylabel('True Positive Rate', fontsize=16)
    # plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
    # plt.legend(loc='lower right', fontsize=13)
    # plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
    # plt.axes().set_aspect('equal')
    # plt.show()

    return result

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [22]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [23]:
def new_connections_predictions():
    
    fc_df = future_connections
    nodes = list(fc_df.index)
    fc_df['pair_node'] = nodes

    #common_neigh
    common_neigh = [len(list(nx.common_neighbors(G,x,y))) for x, y in nodes]
    #Community Common Neighbors
    for x, y in G.nodes(data=True):
        G.node[x]['community'] = G.node[x]['Department']

    #create a function for feature extraction
    def feature_dict(transform_function):
        score = list(transform_function(G))
        score_dict = dict()
        for x,y,z in score:
            if (x,y) not in score_dict.keys():
                score_dict[(x,y)] = z
        return score_dict

    #create score dict
    jar_dict = feature_dict(nx.jaccard_coefficient)
    res_alloc_dict = feature_dict(nx.resource_allocation_index)
    ccn_dict = feature_dict(nx.cn_soundarajan_hopcroft)
    cra_dict = feature_dict(nx.ra_index_soundarajan_hopcroft)
    aai_dict = feature_dict(nx.adamic_adar_index)
    pa_dict = feature_dict(nx.preferential_attachment)

    #create features with 4 feature_dict
    fc_df['common_neighbor'] = common_neigh
    fc_df['jar_score'] = fc_df['pair_node'].map(lambda x: jar_dict[x])
    fc_df['res_alloc_score'] = fc_df['pair_node'].map(lambda x: res_alloc_dict[x])
    fc_df['ccn_score'] = fc_df['pair_node'].map(lambda x: ccn_dict[x])
    fc_df['cra_score'] = fc_df['pair_node'].map(lambda x: cra_dict[x])
    fc_df['pa_score'] = fc_df['pair_node'].map(lambda x: pa_dict[x])
    fc_df['aai_score'] = fc_df['pair_node'].map(lambda x: aai_dict[x])

    #split data into train and test
    test = fc_df[fc_df['Future Connection'].isnull()==1]
    train = fc_df[fc_df['Future Connection'].isnull()==0]

    #look at feature dist for different values of target variable
    #for i in tmp.columns:
        #print(tmp[tmp['Future Connection']==0][i].mean(),(tmp[tmp['Future Connection']==1][i].mean()),i)

    #split train data into X and y
    test_index = fc_df[fc_df['Future Connection'].isnull()]['pair_node']
    train_y = train['Future Connection']
    train_X = train.drop(['pair_node','Future Connection'], axis = 1)
    test_X = test.drop(['pair_node','Future Connection'], axis=1)


#     # #build a model
#     %matplotlib inline
#     from sklearn.ensemble import GradientBoostingClassifier
#     from sklearn.model_selection import train_test_split
#     from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#     from sklearn.metrics import roc_curve, auc
#     import matplotlib.pyplot as plt


#     X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, random_state = 0)
#     gbclf = GradientBoostingClassifier().fit(X_train, y_train)
#     pred_output = gbclf.predict(X_test)
#     y_score_gbclf = gbclf.decision_function(X_test)
#     fpr_gbclf, tpr_gbclf, _ = roc_curve(y_test, y_score_gbclf)
#     roc_auc_gbclf = auc(fpr_gbclf, tpr_gbclf)

#     plt.figure()
#     plt.xlim([-0.01, 1.00])
#     plt.ylim([-0.01, 1.01])
#     plt.plot(fpr_gbclf, tpr_gbclf, lw=3, label='gbclf ROC curve (area = {:0.2f})'.format(roc_auc_gbclf))
#     plt.xlabel('False Positive Rate', fontsize=16)
#     plt.ylabel('True Positive Rate', fontsize=16)
#     plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
#     plt.legend(loc='lower right', fontsize=13)
#     plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
#     plt.axes().set_aspect('equal')
#     plt.show()

    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import roc_curve, auc
    gbclf = GradientBoostingClassifier().fit(train_X, train_y)
    pred_out = gbclf.predict_proba(test_X)

    pred_output = pd.DataFrame(pred_out, index = test_index)

    return pred_output[1]